In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('..')

from backtest.performance_generater import PerformanceGenerator
from backtest.operators import *

In [2]:
open = pd.read_csv('/Volumes/Database/Factor_data/open.csv', index_col=0)
close = pd.read_csv('/Volumes/Database/Factor_data/close.csv', index_col=0)
returns = pd.read_csv('/Volumes/Database/Factor_data/return.csv', index_col=0)

# 要再確認一下 報酬正確性
unadj_return = close.pct_change().shift(-2) 
exp_returns = returns.shift(-2) * 0.01

eps_q = pd.read_csv('/Volumes/Database/Factor_data/季底每股稅後淨利_90.csv', index_col=0)
eps_m = pd.read_csv('/Volumes/Database/Factor_data/每股稅前淨利_90.csv', index_col=0)

/var/folders/fx/jn0gghjd4cg68lr15nlpsbqr0000gn/T/ipykernel_1213/1882419209.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  unadj_return = close.pct_change().shift(-2)


In [8]:
eps_q[-100:]


,1101,1102,1103,1104,1107,1108,1109,1110,1113,1201,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-12,0.41,1.17,0.00,0.62,NaN,NaN,0.60,0.12,NaN,0.07,...,-0.60,1.62,NaN,-0.2,NaN,1.44,-0.02,0.98,0.37,0.33
2023-10-13,0.41,1.17,0.00,0.62,NaN,NaN,0.60,0.12,NaN,0.07,...,-0.60,1.62,NaN,-0.2,NaN,1.44,-0.02,0.98,0.37,0.33
2023-10-16,0.41,1.17,0.00,0.62,NaN,NaN,0.60,0.12,NaN,0.07,...,-0.60,1.62,NaN,-0.2,NaN,1.44,-0.02,0.98,0.37,0.33
2023-10-17,0.41,1.17,0.00,0.62,NaN,NaN,0.60,0.12,NaN,0.07,...,-0.60,1.62,NaN,-0.2,NaN,1.44,-0.02,0.98,0.37,0.33
2023-10-18,0.41,1.17,0.00,0.62,NaN,NaN,0.60,0.12,NaN,0.07,...,-0.60,1.62,NaN,-0.2,NaN,1.44,-0.02,0.98,0.37,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-06,0.16,0.76,0.18,1.42,NaN,0.47,0.34,0.10,NaN,0.12,...,-0.61,2.13,-0.06,-0.2,NaN,1.52,-0.20,1.32,0.97,0.32
2024-03-07,0.16,0.76,0.18,1.42,NaN,0.47,0.34,0.10,NaN,0.12,...,-0.61,2.13,-0.06,-0.2,NaN,1.52,-0.20,1.32,0.97,0.32
2024-03-08,0.16,0.76,0.18,1.42,NaN,0.47,0.34,0.10,NaN,0.12,...,-0.61,2.13,-0.06,-0.2,NaN,1.52,-0.20,1.32,0.97,0.32


In [6]:
eps_m[-100:]

,1101,1102,1103,1104,1107,1108,1109,1110,1113,1201,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2023-12-21,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4
2023-12-22,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4
2023-12-25,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4
2023-12-26,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4
2023-12-27,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4
2023-12-28,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4
2023-12-29,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4
2024-01-02,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4
2024-01-03,0.46,0.99,0.2,1.8,NaN,0.54,0.64,0.14,NaN,0.17,...,-0.06,2.3,-0.03,-0.2,NaN,1.96,-0.2,2.38,1.34,0.4


In [11]:
ZTXA_close = pd.read_csv('/Volumes/Database/Factor_data/ZXTA_close.csv', index_col=0)
ZTXA_close.index = pd.to_datetime(ZTXA_close.index)
market_returns = ZTXA_close['ZTXA'].pct_change()
Benchmark = ZTXA_close['ZTXA'].pct_change().shift(-2)

In [23]:
factor = cs_rank(revenue/close)

# no fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     buy_fee = 0, 
                                                     sell_fee = 0,
                                                     benchmark = Benchmark
                                                     ).backtest()

# fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     benchmark = Benchmark
                                                     ).backtest()

+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   | Turnover |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
| Performance |       389.44 %        |       169.54 %       |     1.14     |    16.57 %     |   31.58 %    |  14.30 %   | 0.90 % |  0.34 %  |
|  Benchmark  |       145.24 %        |       103.31 %       |     0.62     |     9.05 %     |   31.51 %    |  16.17 %   | 1.02 % |   nan    |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+


+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   | Turnover |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+
| Performance |       322.21 %        |       154.76 %       |     1.04     |    14.92 %     |   31.77 %    |  14.30 %   | 0.90 % |  0.34 %  |
|  Benchmark  |       145.24 %        |       103.31 %       |     0.62     |     9.05 %     |   31.51 %    |  16.17 %   | 1.02 % |   nan    |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+----------+


In [19]:
csv_file = '/Volumes/Database/medina_alpha/cs_rank(ps).csv'
factor.to_csv(csv_file)